In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
import json
from datasets import load_dataset

file_path = 'transformed_tfl_theory.json'

alpaca_prompt = """Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.

### Инструкция:
{}

### Ответ:
{}"""

EOS_TOKEN = tokenizer.eos_token


def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    texts = []
    for question, answer in zip(questions, answers):
        text = alpaca_prompt.format(question, answer) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

dataset = load_dataset("json", data_files=file_path, split="train")

dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        #max_steps = 95,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3408,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/150 [00:00<?, ? examples/s]

In [9]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
11.654 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 150 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 90
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.362600
2,2.148100
3,1.823500
4,1.883700
5,1.859600
6,1.814300
7,1.534900
8,1.585900
9,1.485500
10,1.311600


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "язык Дика", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nязык Дика\n\n### Ответ:\nЯзык Дика — это контекстно-свободный язык, то есть язык, в котором линейная комбинация нетерминалов может быть заменена на слово, содержащее терминалы и нетерминалы.\nПример языка Дика: L = {a^kb^mc^n | k, m, n ∈ N}, где L — язык Дика.\nЯзык Дика n + 1: L = {a^kb^mc^n | k, m ∈ N, 0 < n ≤ k + m + 1}, где L — язык Дика n + 1.<|eot_id|>']

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "ЧТо такое теорема Клини", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nЧТо такое теорема Клини\n\n### Ответ:\nТеорема Клини: Язык L регулярен тогда и только тогда, когда он является автоматом. Язык L называется автоматом, если существует конечный автомат, распознающий этот язык.<|eot_id|>']

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "деревья вывода", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nдеревья вывода\n\n### Ответ:\nВыводные деревья — это упорядоченные деревья, в которых каждому узлу назначается символ из алфавита N ∪ Σ. Корень дерева — это начало вывода. Каждый узел, кроме корня, имеет единственную метку. Если A\nузел метки a ∈ N, то к нему добавляется ребро, leading to a узел, метка которого равна a ∈ P∗(w), где w — метка корня\nузел. Узлы меток из алфавита Σ называются листьями дерева. Выходное дерево для строки w ∈\n∗(w) получается путем построения вывода, начинающегося с корня и соответствующего последовательности символов w.\nПримечание. Каждому выходному дереву соответствует уникальная строка w ∈\n∗(w).<|eot_id|>']

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Напиши теорему о несвязном объединении языков LL(k)", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nНапиши теорему о несвязном объединении языков LL(k)\n\n### Ответ:\nТеорема (О непересекающемся объединении языков LL(k): Розенкранц и Стернс [1970]).\nПусть L1,.., Ln ⊆ Σ\n∗ — попарно непересекающиеся языки LL(k) и пусть их объединение\nЛ1 ∪... ∪ Ln регулярно. Тогда все языки L1,..., Ln должны быть регулярны.<|eot_id|>']

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Лемма Ардена", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nЛемма Ардена\n\n### Ответ:\nЛемма Ардена: Язык L регулярен тогда и только тогда, когда он является автоматом.\nЯзык L называется автоматом, если существует конечный автомат, распознающий этот язык.<|eot_id|>']

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "грамматика, данная КС", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nграмматика, данная КС\n\n### Ответ:\nГрамматика, данная КС, — это грамматика без циклов, без лишних символов и без цепочных правил.<|eot_id|>']

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "перечисли свойства регулярных выражений", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nперечисли свойства регулярных выражений\n\n### Ответ:\nРегулярные выражения образуют ассоциативное полукольцо с операциями (0, +, 1, ), то есть для любых\nрегулярных выражений e, f и g выполняются следующие тождества:\n1. e+f = f+e,\n2. e+0 = e,\n3. (e+f)+g = e+(f+g),\n4. e·1 = e,\n5. 1·e = e,\n6. (e·f)·g = e·(f·g),\n7. e·(f+g) = e·f+e·g,\n8. (f+g)·e = f·e+g·e,\n9. e·0=0,\n10. 0·e=0.\nКроме того, для любых регулярных выражений e и f\nвыполняются следующие тождества:\n1. e·0=0,\n2. 0·e=0,\n3. (1+e+ee+... +e^(n−1))(e^n)∗ = e∗ для любого n >= 1,\n4. (e∗f)�']

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "гомоморфизм алфавита", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nгомоморфизм алфавита\n\n### Ответ:\nПусть Σ1 и Σ2 — алфавиты. Если отображение h: Σ∗1 → Σ∗2 удовлетворяет условию h(x y) = h(x) h(y) для всех слов x ∈ Σ∗1 и y ∈ Σ∗1, то h называется\nгомоморфизм алфавита.\nЗамечание: Если h — гомоморфизм алфавита, то h(*) = *.<|eot_id|>']

In [8]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "свойства класса детерминированных контекстно-свободных языков", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nсвойства класса детерминированных контекстно-свободных языков\n\n### Ответ:\nЯзыки DF: если L — это детерминированный контекстно-свободный язык, то\nL∗ также является детерминированным контекстно-свободным языком.\nДобавление пустой строки не образует в общем случае детерминированный контекстно-свободный язык.\nЯзык DF: Если L — это детерминированный контекстно-свободный язык, то\nL^R также является детерминированным контекстно-свободным языком.\nЯзык DF: если L — детерминированный контекстно-свободный язык, то\nL ∩ wR также является детерминированным контекстно-свободным языком.\nЯзык DF: если L — детерминированный контекстно-свободный язык, то\nL U wR также является детерминированным контекстно-свободным языком.\nДобавление ε: если L — детерминированный контекстно-свободный язык, то\nL + wR не является']

In [9]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "гомоморфизм алфавитов", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nгомоморфизм алфавитов\n\n### Ответ:\nПусть Σ1 и Σ2 — алфавиты. Если отображение h: Σ∗1 → Σ∗2 удовлетворяет условию h(x y) = h(x) h(y) для всех слов x ∈ Σ∗1 и y ∈ Σ∗1, то h называется\nгомоморфизмом (морфизмом) алфавита Σ1 в алфавит Σ2.<|eot_id|>']

In [10]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "неконткрактные грамматики", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nнеконткрактные грамматики\n\n### Ответ:\nГрамматика KS — это грамматика без циклов, без лишних символов и без цепных правил.\nЯзык, задаваемый контекстно-свободной грамматикой, называется контекстно-свободным языком.\nГрамматика KS: G = <N, ∑, P, S>, где правила: A → β, где β ∈ (N + ∑)*<|eot_id|>']

In [11]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Напиши свойства регулярных выражений", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nНапиши свойства регулярных выражений\n\n### Ответ:\nРегулярные выражения образуют ассоциативное полукольцо с операциями (0, +, 1, ·), то есть для любых регулярных выражений e, f и g\nвыполняются следующие тождества:\n1. e+f = f+e,\n2. e+0 = e,\n3. (e+f)+g = e+(f+g),\n4. e·1 = e,\n5. 1·e = e,\n6. (e·f)·g = e·(f·g),\n7. e·(f+g) = e·f+e·g,\n8. (f+g)·e = f·e+g·e,\n9. e·0=0,\n10. 0·e=0.\nРавенство понимается как равенство языков, определяемых регулярными выражениями.\nДля любых регулярных выражений e, f и g, если e = f·g, то e = f·g — называется\nправорекурсивная грамматика e в терминах f и g.\nЛемма: Для любых регулярных выражений e,']

In [12]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Определение КС грамматики", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nОпределение КС грамматики\n\n### Ответ:\nCS-грамматика — это грамматика без циклов, без лишних символов и без цепных правил. Язык, задаваемый CS-грамматикой, называется регулярным языком.<|eot_id|>']

In [13]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Что такое левый вывод в КС грамматике", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nЧто такое левый вывод в КС грамматике\n\n### Ответ:\nВывод в КС-грамматике называется левым, если на каждом шаге вывода заменяется самый левый вывод всех вхождений нетерминальных символов.\nЯзык, определяемый левой грамматикой, является регулярным.<|eot_id|>']

In [14]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Бесполезный символ в КС грамматике", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nБесполезный символ в КС грамматике\n\n### Ответ:\nСимвол D (0) бесполезен, если грамматика не имеет вывода вида S → *wDy → *wvy, где v, w, y ∈ ∑*<|eot_id|>']

In [15]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "КС грамматика без циклов", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nКС грамматика без циклов\n\n### Ответ:\nКС грамматика без циклов — это КС грамматика, в которой нет правила: A → +A. То есть грамматика, в которой нет циклов (за исключением возможных циклов в терминальных символах), т.е. нетерминалы не могут быть ими заменены даже после определенного количества шагов.\nПример КС грамматики без циклов:\nS → аАб | ε\nА → б | с<|eot_id|>']

In [16]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "удаление ε-правил", # instruction
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Ниже представлено задание, которое описывает задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nудаление ε-правил\n\n### Ответ:\nε-правила можно удалить, заменив в P всех правил, содержащие ε-правила, на их левые части, а N — на ε.<|eot_id|>']